Dans ce TP, nous considérons des trajets en vélo partagé (similaire au vélib) en Californie. Deux jeux de données sont fournis : l'un qui contient les stations de vélo, l'autre, les trajets à vélo. Les déplacements à vélo se font d'une station à l'autre.

Charger le fichier stationData.csv dans un DataFrame station_df et le fichier tripData.csv dans un DataFrame trip_df. Pour chaque Dataframe, il vous faudra demander une inférence des schémas et indiquer que la première ligne est un en-tête.

In [0]:
%fs ls /FileStore/tables

path,name,size,modificationTime
dbfs:/FileStore/tables/stationData.csv,stationData.csv,5272,1732186175000
dbfs:/FileStore/tables/tripData.csv,tripData.csv,43012642,1732186272000


In [0]:
station_df = spark.read.csv("/FileStore/tables/stationData.csv", inferSchema=True, header=True)
trip_df = spark.read.option("inferSchema","true").csv("/FileStore/tables/tripData.csv",header=True)

Afficher les schémas des 2 DataFrames.

In [0]:
trip_df.printSchema()

station_df.printSchema()

root
 |-- TripID: integer (nullable = true)
 |-- Duration: integer (nullable = true)
 |-- StartDate: string (nullable = true)
 |-- StartStation: string (nullable = true)
 |-- StartTerminal: integer (nullable = true)
 |-- EndDate: string (nullable = true)
 |-- EndStation: string (nullable = true)
 |-- EndTerminal: integer (nullable = true)
 |-- Bike#: integer (nullable = true)
 |-- SubscriberType: string (nullable = true)
 |-- ZipCode: string (nullable = true)

root
 |-- station_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- dockcount: integer (nullable = true)
 |-- landmark: string (nullable = true)
 |-- installation: date (nullable = true)



Créer une vue pour chaque DataFrame.

In [0]:
# Votre code ici
#display(trip_df)

station_df.createOrReplaceTempView("station")
trip_df.createOrReplaceTempView("trip")

Trouver deux façons de calculer le nombre de trajets, l'une en appelant une méthode sur trip_df directement, l'autre en rédigeant une requête SQL de la vue correspondant au DataFrame tripData.

In [0]:
# Votre code ici
trip_df.count()

spark.sql("select count(1) from trip").show()

+--------+
|count(1)|
+--------+
|  354152|
+--------+



Ecrire une requête permettant de compter le nombre de trajets qui démarrent et se terminent à la même station.

In [0]:
# Votre code ici
spark.sql("select count(1) from trip where StartTerminal=EndTerminal").show()

#trip_df.filter(trip_df['StartTerminal']==trip_df['EndTerminal']).count()

+--------+
|count(1)|
+--------+
|   10276|
+--------+



On souhaite désormais obtenir l’id des stations associées à ces trajets. Ecrire une requête renvoyant la liste des terminaux concernés ainsi que le nombre de trajets pour chacun de ces terminaux. Trier le résultat par ordre décroissant de nombre de trajets.
<br>Exemple de sortie :
<br>+--------+--------+
<br>|terminal|count(1)|
<br>+--------+--------+
<br>| 60| 850|
<br>| 50| 708|
<br>| 35| 348|
<br>| 76| 320|
<br>| 74| 307|
<br>(La station 60 est la plus concernée par ces trajets cycliques, avec 850 de ces trajets.)

In [0]:
# Votre code ici
spark.sql("select StartTerminal, count(1) from trip where StartTerminal=EndTerminal group by StartTerminal order by 2 desc;").show()

#trip_df.filter(trip_df['StartTerminal']==trip_df['EndTerminal']).groupBy(trip_df['StartTerminal']).count(orderBy(desc("count"))).show()

+-------------+--------+
|StartTerminal|count(1)|
+-------------+--------+
|           60|     850|
|           50|     708|
|           35|     348|
|           76|     320|
|           74|     307|
|           39|     296|
|           61|     280|
|           67|     277|
|           71|     268|
|           70|     260|
|           28|     254|
|           48|     248|
|           54|     230|
|           69|     227|
|           42|     213|
|           73|     200|
|           57|     197|
|           64|     194|
|            3|     189|
|           72|     181|
+-------------+--------+
only showing top 20 rows



Dans la requête précédente, nous avons oublié un élément qui nous importe. Nous souhaitons compléter le résultat en indiquant le nombre de docks (dockcount) des stations concernées.
<br>Exemple de sortie :
<br>+--------+---------+--------+
<br>|terminal|dockcount|count(1)|
<br>+--------+---------+--------+
<br>| 60| 15| 850|
<br>| 50| 23| 708|
<br>| 35| 11| 348|
<br>| 76| 19| 320|
<br>| 74| 23| 307|
<br>Mettre à jour la requête.

In [0]:
# Votre code ici
spark.sql(
    """
    select StartTerminal, dockcount, count(1) from trip, station where StartTerminal=EndTerminal and StartTerminal=station_id group by StartTerminal,dockcount order by 3 desc;
    """).show()

+-------------+---------+--------+
|StartTerminal|dockcount|count(1)|
+-------------+---------+--------+
|           60|       15|     850|
|           50|       23|     708|
|           35|       11|     348|
|           76|       19|     320|
|           74|       23|     307|
|           39|       19|     296|
|           61|       27|     280|
|           67|       27|     277|
|           71|       19|     268|
|           70|       19|     260|
|           28|       23|     254|
|           48|       15|     248|
|           54|       15|     230|
|           69|       23|     227|
|           42|       15|     213|
|           73|       15|     200|
|           57|       15|     197|
|           64|       15|     194|
|            3|       15|     189|
|           72|       23|     181|
+-------------+---------+--------+
only showing top 20 rows



Rédiger les 2 requêtes précédentes avec le DSL de DataFrame.

In [0]:
from pyspark.sql.functions import col, desc


#trip_df.filter(trip_df['StartTerminal']==trip_df['EndTerminal']).groupBy(trip_df['StartTerminal']).count(orderBy(desc("count"))).show()


res = trip_df.join(
    station_df,
    on=col("StartTerminal")==col("station_id"),
    how="left"
).filter(col("StartTerminal")==col("EndTerminal")
).groupBy(
    "StartTerminal",
    "dockcount",
).count(
).orderBy(
    desc("count")
)

res.show()

+-------------+---------+-----+
|StartTerminal|dockcount|count|
+-------------+---------+-----+
|           60|       15|  850|
|           50|       23|  708|
|           35|       11|  348|
|           76|       19|  320|
|           74|       23|  307|
|           39|       19|  296|
|           61|       27|  280|
|           67|       27|  277|
|           71|       19|  268|
|           70|       19|  260|
|           28|       23|  254|
|           48|       15|  248|
|           54|       15|  230|
|           69|       23|  227|
|           42|       15|  213|
|           73|       15|  200|
|           57|       15|  197|
|           64|       15|  194|
|            3|       15|  189|
|           72|       23|  181|
+-------------+---------+-----+
only showing top 20 rows



In [0]:
# Votre code ici

Observer le plan d’exécution des requêtes.

In [0]:
# Votre code ici
res.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [count#1178L DESC NULLS LAST], true, 0
   +- Exchange rangepartitioning(count#1178L DESC NULLS LAST, 200), ENSURE_REQUIREMENTS, [plan_id=2565]
      +- Project [StartTerminal#273, dockcount#242, count(1)#1177L AS count#1178L]
         +- HashAggregate(keys=[StartTerminal#273, dockcount#242], functions=[finalmerge_count(merge count#1192L) AS count(1)#1177L])
            +- Exchange hashpartitioning(StartTerminal#273, dockcount#242, 200), ENSURE_REQUIREMENTS, [plan_id=2561]
               +- HashAggregate(keys=[StartTerminal#273, dockcount#242], functions=[merge_count(merge count#1192L) AS count#1192L])
                  +- Project [StartTerminal#273, dockcount#242, count#1192L]
                     +- BroadcastHashJoin [StartTerminal#273], [station_id#238], LeftOuter, BuildRight, false, true
                        :- HashAggregate(keys=[StartTerminal#273], functions=[partial_count(1) AS count#1192L])
                      